# 作业1

预测PM2.5值

## 1 数据预处理

### 1.1 导入训练数据

处理源数据，去掉不要的前3列，将降雨特征中的NR改成0，并将数据转化为numpy类型

### 1.2 将数据转化为18维特征和label

本来的表格 4320*24 每18行一天共480天，一天24个数据  
处理成连续的 12*18*480   
12个月，18维输入，每个月20天*24小时=480个数据

### 1.3 滑动窗口获得471个样本，并整合样本格式(162,1)
根据前9小时的数据预测第10小时的数据,  
每个月的20天都是连续的,  
每个月可以获得的sample：20\*24-10+1=471,  
12个月一共有471\*12=5652个sample,  
每个sample中,  
data由前9小时的18维数据组成，共9\*18=162个数据,  
label是第10小时的PM2.5值,  
下面处理把month_data处理成dataset和labels  

### 1.4 归一化
归一化，采用均值方差  
$ \frac{(x-\mu )}{S} $ 其中 $\mu$是均值，$S$是标准差  
抽取80%为训练集，20%为验证集

In [1]:
'''1.1'''
import sys
import pandas as pd
import numpy as np
import math

#导入训练数据
data=pd.read_csv('E:/dl2021/DL-hw/hw1/train.csv',encoding='big5')

#删掉前面3列的无效数据
data=data.iloc[:,3:]
#print(data)

#数据清洗，把NR换成0
data.replace('NR',0,inplace = True)

#转换成numpy数据类型
raw_data=data.to_numpy()
#print(raw_data)

'''1.2'''
month_data={}
for month in range(12):
    sample=np.empty([18,480])
    for day in range(20):
        sample[:,day*24:(day+1)*24]=raw_data[(day+month*20)*18:(day+1+month*20)*18,:]
    month_data[month]=sample

#print(month_data[0])

'''1.3'''
x=np.empty([471*12,9*18],dtype=float)
y=np.empty([471*12,1],dtype=float)
for month in range(12):
    for hour in range(471):
        x[month*471+hour,:]=month_data[month][:,hour:hour+9].reshape(-1)
        y[month*471+hour]=month_data[month][9,hour+9]

# print(x[1])

'''1.4'''
#归一化
#(x-μ)/S
x_mean=np.mean(x,axis=0)
x_std=np.std(x,axis=0)
for i in range(len(x)):
    for j in range(len(x[0])):
        if x_std[j]!=0:
            x[i][j]=(x[i][j]-x_mean[j])/x_std[j]

'''
#从训练集里面20%作为验证集
div=math.floor(len(x)*0.8)
x_train_set=x[:div,:]
y_train_set=y[:div,:]
x_valid_set=x[div:,:]
y_vaild_set=y[div:,:]
print(y_train_set)
print(y_vaild_set)

x_train_set.shape
'''
x_train_set=x
y_train_set=y

## 2 处理测试集
处理测试集，每18行是一天，共239天，根据一天的前9小时数据预测第10小时

In [2]:
#导入测试数据
data_test=pd.read_csv('E:/dl2021/DL-hw/hw1/test.csv',header=None,encoding='big5')

#删掉前面2列的无效数据
data_test=data_test.iloc[:,2:]
#print(data)

#数据清洗，把NR换成0
data_test.replace('NR',0,inplace = True)

#转换成numpy数据类型
raw_test=data_test.to_numpy()
#print(raw_test)
#print(raw_test.shape)

#把240个样本转换成18*9=162维
x_test_set=np.empty([240,162],dtype=float)
for i in range(240):
    x_test_set[i,:]=raw_test[i*18:i*18+18,:].reshape(-1)

#print(x_test_set)

#归一化
#(x-μ)/S
x_test_mean=np.mean(x_test_set,axis=0)
x_test_std=np.std(x_test_set,axis=0)
#print(x_test_mean.shape,x_test_std.shape,x_test_set.shape)


for i in range(240):
    for j in range(162):
        if x_test_std[j]!=0:
            x_test_set[i][j]=(x_test_set[i][j]-x_test_mean[j])/x_test_std[j]
            
#print(x_test_set)

#拼接test_set，维度为（240，163），即在补上bias结点，默认特征为1
x_test_set=np.concatenate(
    (np.ones([len(x_test_set),1]),x_test_set),axis=1).astype(float)

#print(x_test_set)

___

# 2 training开始训练
确定超参数：步长（学习速率）、迭代次数  

In [3]:
#确定超参数
#维度dimention，18维*9天的数据+1个bias=163
dim=18*9+1
#初始化w，维度为 (163,1)
w=np.zeros([dim,1])
#拼接training_set，维度为（4521，163），即在补上bias结点，默认特征为1
x_train_set=np.concatenate(
    (np.ones([len(x_train_set),1]),x_train_set),axis=1).astype(float)
#设置学习率
learning_rate=1
learning_lambda=1
#设置轮数
epoch=1

#初始化movement为0
movement=[]
movement.append(np.zeros([dim,1]))

#x_train_set.shape

## 计算损失函数 L
采用均方根误差，是预测值$wx$与真实值$y$偏差的平方与观测次数$num$比值的平方根  
$L^{t}=\left(\hat{y}^{t}-\left(b+\sum w_{i} x_{i}^{t}\right)\right)^{2}$


## 计算梯度 gradient
$\frac{\partial L}{\partial w}=-2Lx_i$

## 梯度下降法迭代权重
$\mathrm{v}^{t+1}=\lambda \mathrm{v}^{t}-\eta \nabla L\left(\theta^{t}\right)$  
$\theta^{t+1}=\theta^{t}+\mathrm{v}^{t+1}$

In [4]:
#训练
w=('weight.npy')
for e in range(epoch):
    for t in range(len(x_train_set)):
        #计算损失函数 sample(1,163) W(163,1)
        loss=np.power(y_train_set[t]-np.dot(x_train_set[t],w),2)
        if(t%5==0):
        #每5个time step输出一次结果
            print("迭代次数{}:Loss函数值{}".format(t,loss[0]))
        #计算gradient
        gradient=-2*loss*x_train_set[t]
        #计算movement
        movement.append(learning_lambda*movement[t]-learning_rate*gradient)
        #迭代
        w=w+movement[t+1]
        

TypeError: Cannot cast array data from dtype('float64') to dtype('<U32') according to the rule 'safe'

In [1]:
np.save('weight.npy', w)

NameError: name 'np' is not defined

## 3 预测测试集

训练完成，接下来在验证集上验证模型
将预测结果存储到CSV文件

In [6]:
#预测
predict_y=np.dot(x_test_set,w)
#print(predict_y)

#将预测结果存储到CSV文件
import csv
with open('submit.csv',mode='w',newline='') as submit_file:
    csv_writer=csv.writer(submit_file)
    header=['id','value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row=['id_'+str(i),predict_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id', 'value']
['id_0', 4.292833067456147]
['id_1', 14.373494656768624]
['id_2', 17.721634429236296]
['id_3', 5.357966374417984]
['id_4', 20.82147838420883]
['id_5', 16.555737971561037]
['id_6', 17.822219583942587]
['id_7', 23.49073371397133]
['id_8', 13.03798641976596]
['id_9', 46.92257300225636]
['id_10', 8.709727680165948]
['id_11', 6.596968414537493]
['id_12', 49.18099903077797]
['id_13', 41.186255969411214]
['id_14', 16.38641995299686]
['id_15', 8.895647160617575]
['id_16', 24.991910988233876]
['id_17', 51.74389861704947]
['id_18', -2.247078051225884]
['id_19', 12.527906968919755]
['id_20', 32.69374037410227]
['id_21', 56.18145315669767]
['id_22', 7.0343268406175135]
['id_23', 13.825878348301844]
['id_24', 10.967055483192446]
['id_25', 29.11090725490343]
['id_26', 11.17707986109438]
['id_27', 50.88766667569088]
['id_28', 4.814494651288271]
['id_29', 43.50468356594277]
['id_30', 18.68369496349019]
['id_31', 5.451466341979973]
['id_32', 2.3915266075820227]
['id_33', 13.753820398808

___
## 4 模型评估

```
w=np.load('weight.npy')
#拼接training_set，维度为（1131，163），即在补上bias结点，默认特征为1
x_valid_set=np.concatenate(
    (np.ones([len(x_valid_set),1]),x_valid_set),axis=1).astype(float)
x_valid_set.shape

#计算损失函数
y_valid_predict=np.dot(x_valid_set,w)
loss_valid=np.sqrt(
    np.sum(np.power(y_valid_predict-y_vaild_set,2))/len(x_valid_set))
print("the Loss on valid set is {}".format(loss_valid))
for i in range(20):
    print("预测值为：{}，真实值为：{}".format(y_valid_predict[i],y_vaild_set[i]))
```